In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [2]:
!pip3 install  --upgrade bitsandbytes  --upgrade peft  --upgrade transformers --upgrade datasets --upgrade accelerate --upgrade trl

In [4]:
!pip install accelerate wandb --upgrade

In [1]:
!pip install jupyter --upgrade ipywidgets --upgrade


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install matplotlib seaborn captum

     ---------------------------------------- 0.0/165.9 kB ? eta -:--:--
     --------- ------------------------------ 41.0/165.9 kB ? eta -:--:--
     -------------------------------------- 165.9/165.9 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.0 MB 5.9 MB/s eta 0:00:02
   --- ------------------------------------ 0.8/8.0 MB 8.2 MB/s eta 0:00:01
   ------ --------------------------------- 1.2/8.0 MB 8.6 MB/s eta 0:00:01
   -------- ------------------------------- 1.7/8.0 MB 9.9 MB/s eta 0:00:01
   --------- ------------------------------ 1.9/8.0 MB 9.5 MB/s eta 0:00:01
   --------- ------------------------------ 1.9/8.0 MB 9.5 MB/s eta 0:00:01
   --------- ------------------------------ 1.9/8.0 MB 9.5 MB/s eta 0:00:01
   ------------- -------------------------- 2.6/8.0 MB 6.6 MB/s eta 0:00:01
   ---------------- ----------------------- 3.2/8.0 MB 7.6 MB/s eta 0:00:01
   -----------------


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import time
from random import randrange, sample, seed

import torch
from datasets import load_dataset
import wandb
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, EarlyStoppingCallback
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from accelerate import Accelerator, PartialState

In [2]:
#from kaggle_secrets import UserSecretsClient
import os

#user_secrets = UserSecretsClient()
os.environ["HF_TOKEN"] = ''
os.environ["WANDB_PROJECT"] = "Final_Comparison_128_256"
#os.environ["WANDB_PROJECT"] = "test llm prompt"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [3]:
wandb.login(key ='')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ashwinksharma01. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ccbdp\_netrc


True

In [4]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct" 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model in full precision for training
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_cache=False,
    quantization_config=bnb_config,
    device_map='auto',
    token=os.environ["HF_TOKEN"],
    attn_implementation='eager'
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

c:\Users\ccbdp\Music\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ccbdp\.cache\huggingface\hub\models--meta-llama--Meta-Llama-3-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token = os.environ["HF_TOKEN"], # if model is gated like llama or mistral
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [6]:
# Load dataset from the hub
# dataset = load_dataset("csv", data_files ="spider-old/CSV/final.csv",split='train')
#dataset = dataset.select(range(750))

dataset = load_dataset("csv", data_files ="wiki_postgres/train.csv",split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
# eval_dataset = load_dataset("csv", data_files="spider-old/CSV/final_validation.csv",split='train')
#eval_dataset = eval_dataset.select(range(20))
eval_dataset = load_dataset("csv", data_files="wiki_postgres/test.csv",split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
len(dataset)

56312

In [9]:
dataset

Dataset({
    features: ['question', 'create_table_statement', 'sql_query', 'wiki_sql_table_id'],
    num_rows: 56312
})

In [10]:
# def format_instruction(sample):
# 	output_texts = []
# 	for i in range(len(sample['query'])):
# 		text = f"""### Instruction:
# You are an SQL query generator. Convert the natural language question into an SQL query based on the given database schema.
		
# Your task is to:
# 1. Understand the information requested.
# 2. Identify relevant tables and columns from the schema.
# 3. Construct an accurate and efficient SQL query.

# Output a valid SQL query or return 'I DO NOT KNOW' if the question can't be answered

# ### Database schema:
# {sample['Schema (values (type))'][i]}

# ### Natural language question:
# {sample['question'][i]}

# ### SQL:
# {sample['query'][i]}
# """
# 		output_texts.append(text)
	
# 	return output_texts

def format_instruction(sample):
	output_texts = []
	for i in range(len(sample['sql_query'])):
		text = f"""### Instruction:
You are an SQL query generator. Convert the natural language question into an SQL query based on the given database schema.

Your task is to:
1. Understand the information requested.
2. Identify relevant tables and columns from the schema.
3. Construct an accurate and efficient SQL query.

Output a valid SQL query or return 'I DO NOT KNOW' if the question can't be answered

### Database schema:

{sample['create_table_statement'][i]}

### Natural language question:

{sample['question'][i]}

### SQL:
{sample['sql_query'][i]}
"""
		output_texts.append(text)

	return output_texts

In [11]:
# dataset['query'][1]

In [12]:
# dataset['Schema (values (type))'][1]

In [13]:
response_template = "### SQL:\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
#collator = DataCollatorForCompletionOnlyLM(tokenizer.encode(response_template, add_special_tokens = False)[2:], tokenizer=tokenizer)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model.config.pretraining_tp = 1
model.named_parameters()

<generator object Module.named_parameters at 0x0000025CFC7E2040>

In [14]:
peft_config = LoraConfig(
lora_alpha=128,
lora_dropout=0.1,
r=256, #(scale factor = alpha/r) 
#use_rslora = True, #(should we use ?) ---> (scale factor = alpha/sqrt(r)) r=64, alpha = 16 or 8
bias="none",
task_type="CAUSAL_LM",
target_modules=[
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj",
],
)   

model = prepare_model_for_kbit_training(model)

In [15]:
args = TrainingArguments(
    output_dir="llama-3-8B-it-4bit-sql_wiki_steps_old_prompt_128_256_fixed_data",
    #num_train_epochs=3,  
    per_device_train_batch_size=1,  # Adjust batch size
    per_device_eval_batch_size=1,
    # auto_find_batch_size=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    load_best_model_at_end=True,
    optim='adamw_torch',  
    logging_steps=300,
    save_strategy="steps",  
    evaluation_strategy="steps",  
    save_steps=300, 
    eval_steps=300,  
    learning_rate=5e-5,
    weight_decay=0.3,
    bf16=False, 
    fp16=False,  
    tf32=False,  
    max_grad_norm=0.7,
    warmup_ratio=0.3,
    lr_scheduler_type='linear',
    disable_tqdm=False,
    report_to='wandb',
    )

c:\Users\ccbdp\Music\.venv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
)

In [17]:
model = get_peft_model(model, peft_config)
    
# trainer instance
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,  
    eval_dataset = eval_dataset,
    peft_config=peft_config,
    max_seq_length=4096,
    tokenizer=tokenizer,
    packing=False,
    formatting_func=format_instruction, 
    data_collator=collator,
    args=args,
    callbacks=[early_stopping],
)

c:\Users\ccbdp\Music\.venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\ccbdp\Music\.venv\Lib\site-packages\trl\trainer\sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/56312 [00:00<?, ? examples/s]

Map:   0%|          | 0/15871 [00:00<?, ? examples/s]

In [18]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/42234 [00:00<?, ?it/s]

c:\Users\ccbdp\Music\.venv\Lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.3586, 'grad_norm': 1.248368501663208, 'learning_rate': 1.1838055402099282e-06, 'epoch': 0.02}


  0%|          | 0/15871 [00:00<?, ?it/s]

...

wandb: Network error resolved after 0:03:21.184328, resuming normal operation.


{'eval_loss': 0.04381299763917923, 'eval_runtime': 9581.9095, 'eval_samples_per_second': 1.656, 'eval_steps_per_second': 1.656, 'epoch': 0.28}


wandb: Adding directory to artifact (.\llama-3-8B-it-4bit-sql_wiki_steps_old_prompt_128_256_fixed_data\checkpoint-3900)... Done. 52.2s
c:\Users\ccbdp\Music\.venv\Lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
wandb: Network error resolved after 0:01:38.409386, resuming normal operation.
wandb: Network error resolved after 0:01:36.929099, resuming normal operation.
wandb: Network error resolved after 0:16:53.674619, resuming normal operation.


{'loss': 0.0468, 'grad_norm': 0.39638301730155945, 'learning_rate': 1.6573277562938998e-05, 'epoch': 0.3}


  0%|          | 0/15871 [00:00<?, ?it/s]

In [ ]:
wandb.finish()
